In [ ]:
import numpy as np, json
from pathlib import Path
import matplotlib
# matplotlib.use("TkAgg")   # Non-interactive backend (saves figures to file)
from matplotlib import pyplot as plt
import plotly.express as px
import pandas as pd

## Load sweep results

In [ ]:
root = Path("/Users/nick/Data/symmetry/sweeps/")
output_dir = root / "sweep02_jax_phase_space"


# Reload arrays
data = np.load(output_dir / "results.npz")
N = data["N"]
L = data["L"]

# Reload parameters
with open(output_dir / "params.json") as f:
    param_dicts = json.load(f)

# print(N.shape, L.shape, len(param_dicts))
param_df = pd.DataFrame.from_dict(param_dicts)
param_df.head()

## Plot example

In [ ]:
sim_idx = 0  # <-- change this
sample_data = np.array(N[sim_idx])  # shape (time, space)

# imshow expects (rows, cols) = (time, space)

fig = px.imshow(
    sample_data.T,                # transpose so x=time, y=space
    aspect="auto",
    origin="lower",
    labels=dict(x="Time", y="Position", color="Intensity"),
    color_continuous_scale="viridis"
)
fig.show()

In [ ]:
import plotly.graph_objects as go

plot_times = np.arange(0, 121,10)
x_grid=np.linspace(-750,750, 151)
fig = go.Figure()
for p in plot_times:
    fig.add_traces(go.Scatter(x=x_grid, y=sample_data[p]))
fig.show()

### Question 1: what dictates survival/extintion of initial Nodal blip?
To start consider case when K_N = N_nl

In [ ]:
x_samp = 61
t_samp = -1
K_A = 667

N_vec = N[:, t_samp, x_samp].ravel() / K_A 
N_vec_e = N[:, t_samp, 0].ravel() / K_A
L_vec = L[:, t_samp, x_samp].ravel() / K_A
kappa_filter = param_df["kappa_NL"].astype(int)==1
# fig = px.scatter(x=N_vec[kappa_filter], y=L_vec[kappa_filter], color=np.log10(rat_vec[kappa_filter]))
fig = px.scatter(x=N_vec[:], y=L_vec[:], color=np.log10(param_df.loc[:, "kappa_NL"]))
    #np.where(kappa_filter)[0])#np.log10(param_df.loc[kappa_filter, "a_amp"]+1e-6))
fig.show()

In [ ]:
# rat_vec = np.divide(L_vec, N_vec)
# nz_ft = N_vec > 2
# fig = px.scatter(x=param_df.loc[kappa_filter & nz_ft, "beta_r"], y=rat_vec[kappa_filter & nz_ft], trendline="ols")
# fig.show()

In [ ]:
import numpy as np
from scipy.optimize import curve_fit
from tqdm import tqdm

def gauss(x, A, mu, sigma):
    return A * np.exp(-(x-mu)**2/(2*sigma**2))

In [ ]:
peaked_inds = np.where((N_vec-N_vec_e)>1)[0]
t_samp = -1
amp_vec = np.empty_like(N_vec)
amp_vec[:] = np.nan
sigma_vec = np.empty_like(N_vec)
sigma_vec[:] = np.nan

for i in tqdm(peaked_inds):
    A = np.squeeze(N[i, t_samp, :])
    popt, _ = curve_fit(gauss, x_grid, A, p0=[A.max(), 0, 100.0])
    amp_vec[i], _, sigma_vec[i] = popt

In [ ]:
lambda_ = np.sqrt(1.85/1e-4)
fig = px.scatter(x=amp_vec/K_A, y=sigma_vec, color=np.log10(param_df["beta_r"] / 15))

fig.add_hline(y=1, line_dash="dash", line_color="black") 

fig.show()

## Find simplest conditions that still support stable domain formation
Can beta_r=1? What range of deltas is permissible? Start with simplest case and add degrees of freedom as needed

In [ ]:
N_array = np.squeeze(N[:, t_samp, :])
N_p = np.divide(N_array, np.sum(N_array, axis=1)[:, None])
N_ent = -np.sum(np.multiply(N_p, np.log(N_p+1e-12)), axis=1)

In [ ]:
mu_N = 1e-4
K_A = 667
beta_a = 1 / (mu_N * K_A)
beta_a

In [ ]:
kappa_filter = param_df["kappa_NL"].astype(int)==1
beta_filter = np.round(param_df["beta_r"]).astype(int)==15 # beta_r=beta_a
plot_filter = (kappa_filter & beta_filter).to_numpy()

fig = px.scatter(x=np.log10(param_df["kappa_NL"]), y=N_ent )

# fig.add_hline(y=1, line_dash="dash", line_color="black") 

fig.show()

In [ ]:
fig = px.scatter(x=np.log10(param_df["beta_r"] / 15), y=N_ent)

fig.show()

In [ ]:
fig = px.scatter(x=np.log10(param_df["delta"]), y=N_ent)
fig.show()

In [ ]:
fig = px.scatter(x=np.log10(param_df["kappa_NL"]), y=np.log10(param_df["beta_r"]/15), size=1+N_vec - N_vec_e)
fig.show()

In [ ]:
fig = px.scatter(x=sigma_vec, y=amp_vec/K_A, o)
fig.show()


Looks like $K_{NL}=10 K_N$, $\beta_r=10 \beta_A$, $D_L/D_N=10$ is a good sweet spot